# Subword tokenization

> Subword tokenization algorithms rely on the principle that **frequently used words should not be split into smaller subwords**, but **rare words should be decomposed
> into meaningful subwords**. 

For instance "annoyingly" might be considered a rare word and could be decomposed into "annoying" and "ly". Both "annoying" and "ly" as stand-alone subwords would appear more frequently while at the same time the meaning of "annoyingly" is kept by the composite meaning of "annoying" and "ly". 

## Byte pair encoding (BPE)

- BPE relies on a pre-tokenizer that splits the training data into words
- After pre-tokenization, a set of unique words has been created and the frequency with which each word occurred in the training data has been determined
- Next, BPE creates a base vocabulary consisting of all symbols that occur in the set of unique words and learns merge rules to form a new symbol from two symbols of the base vocabulary

Example:

- let’s assume that after pre-tokenization, the following set of words including their frequency has been determined:

`("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)`

- Splitting all words into symbols of the base vocabulary, we obtain:

`("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)`

- BPE then counts the frequency of each possible **symbol pair** and picks the symbol pair that occurs **most frequently**. 

In the example above `"h"` followed by `"u"` is present `10 + 5 = 15 times` (10 times in the 10 occurrences of "hug", 5 times in the 5 occurrences of "hugs").

However, the most frequent symbol pair is `"u"` followed by `"g"`, occurring `10 + 5 + 5 = 20 times` in total.

Thus, the first merge rule the tokenizer learns is to group all `"u"` symbols followed by a `"g"` symbol together. Next, `"ug"` is added to the vocabulary. The set of words then becomes:

`("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)`

- BPE then identifies the next most common symbol pair. It’s `"u"` followed by `"n"`, which occurs 16 times. `"u"`, `"n"` is merged to `"un"` and added to the vocabulary. The next most frequent symbol pair is `"h"` followed by `"ug"`, occurring 15 times. Again the pair is merged and `"hug"` can be added to the vocabulary.

`("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)`

Assuming, that the Byte-Pair Encoding training would stop at this point, the learned merge rules would then be applied to new words 

For instance, the word `"bug"` would be tokenized to `["b", "ug"]` but `"mug"` would be tokenized as `["<unk>", "ug"]` since the symbol `"m"` is not in the base vocabulary.

- the vocabulary size, i.e. the **base vocabulary size** + **the number of merges**, is a hyperparameter to choose. 

For instance GPT has a vocabulary size of `40,478` since they have `478` base characters and chose to stop training after `40,000` merges.





## WordPiece

- WordPiece is the subword tokenization algorithm used for BERT, DistilBERT, and Electra.
- The algorithm is the same with BPE but there is a difference:
  - WordPiece does not choose the most frequent symbol pair, but the one that **maximizes the likelihood** of the training data once added to the vocabulary.

 E.g. `"u"`, followed by `"g"` would have only been merged if the probability of `"ug"` divided by `"u"`, `"g"` would have been greater than for any other symbol pair.

## Unigram

- In contrast to **BPE** or **WordPiece**, **Unigram** initializes its base vocabulary to a large number of symbols and progressively trims down each symbol to obtain a smaller vocabulary. 
- `Unigram` is not used directly for any of the models in the transformers, but it’s used in conjunction with `SentencePiece`.
- At each training step, the `Unigram` algorithm defines a loss (often defined as the log-likelihood) over the training data given the current vocabulary and a unigram language model. 
- Then, for each symbol in the vocabulary, the algorithm computes `how much the overall loss would increase` if the symbol was to be removed from the vocabulary. 
- Unigram then removes `p` (with `p` usually being `10%` or `20%`) percent of the symbols whose loss increase is the lowest
- This process is repeated until the vocabulary has reached the desired size. 

## SentencePiece

- All tokenization algorithms described so far have the same problem: It is assumed that the input text uses **spaces** to separate words.
- However, not all languages use spaces to separate words.
- One possible solution is to use language specific pre-tokenizers, e.g. XLM uses a specific Chinese, Japanese, and Thai pre-tokenizer).

=> To solve this problem more generally, SentencePiece treats the input as a raw input stream, thus including the space in the set of characters to use.
It then uses the BPE or unigram algorithm to construct the appropriate vocabulary.
